### DTS201TC 
#### individual Project

In [1]:
import numpy as np 
import scipy.io
import scipy.stats
import math
import collections

#### Auxiliary functions

In [2]:
def log_normpdf(x, mu, sigma):
    """
      Computes the natural logarithm of the normal probability density function
  
    """
    prob = scipy.stats.norm(mu, sigma).pdf(x)
    return prob


In [3]:
class JointParts():
    def __init__(self, N_class):
        self.means = np.zeros([3,N_class])
        self.sigma = np.zeros([3,N_class])


In [4]:
class Model():
    def __init__(self, N_class, N_joints):
        self.jointparts = [JointParts(N_class) for i in range(N_joints)]
        self.class_priors = {}
        

In [5]:
def cal_acc(a,b):
    """
       Input
           two vectors with same size

       Output
           accuracy
    """
    
    acc = 0.0
    for i in range(len(a)):
        if(a[i]==b[i]):
            acc+=1
    acc = acc/len(a)
    
    return acc

#### Functions to implement

In [6]:
def fit_model(X):
    """
      Compute the mean and standard deviation of X, 
    """
    # TODO
    X = np.array(X)
    mean = np.mean(X)
    std = np.sqrt(np.dot(X - mean, (X - mean).T) / X.shape[0])
    # print(mean, variance)
    return (mean, std)


In [7]:
def learn_model(dataset, labels, G=None):
    """
    Input:
     dataset: The data as loaded 
     labels:  The labels as loaded 
     
    Output: the model
     a (tentative) structure for the output model is:
       model.class_priors: containing a vector with the prior estimations
                           for each class
       model.jointparts[i] contains the estimated parameters for the i-th joint

            model.jointparts(i).means: a matrix of 3 x #classes with the
                   estimated means for each of the x,y,z variables of the 
                   i-th joint and for each class.
            model.jointparts(i).sigma: a matrix of 3 x #classes with the
                   estimated stadar deviations for each of the x,y,z 
                   variables of the i-th joint and for each class.

       

    """
    # Get the basic information from the dataset
    label = np.unique(labels)
    labels = labels.flatten()
    N_class = len(label)
    N_joints = dataset.shape[0]
    N_dims = dataset.shape[1]
    model = Model(N_class, N_joints)

    # Prior probability
    cnt = collections.Counter(labels)
    class_items = {}
    for k,v in cnt.items():
        model.class_priors[k] = v/len(labels)
        class_items[k] = []
    
    for i in range(len(labels)):
        class_items[labels[i]].append(i)

    # Posterior probability
    for i in range(N_joints):
        for j in range(N_dims):
            for k in range(N_class):
                data = [dataset[i][j][idx] for idx in class_items[label[k]]]
                (model.jointparts[i].means[j][k], model.jointparts[i].sigma[j][k]) = fit_model(data)
    return model

In [8]:
def classify_samples(instances, model):
    """    
    Input
       instance: a 20x3x#instances matrix defining body positions of
                 instances
       model: as the output of learn_model

    Output
       probs: a matrix of #instances x #classes with the probability of each
              instance of belonging to each of the classes

    Important: to avoid underflow numerical issues this computations should
               be performed in log space
    """
    # TODO
    # Get the information of dataset from model
    
    label = list(model.class_priors.keys())
    N_class = len(label)
    N_joints = len(model.jointparts)
    N_dims = model.jointparts[0].means.shape[0]
    
    # Make sure the model is suitable for the test dataset
    assert instances.shape[0] == N_joints, instances.shape[1] == N_dims

    # Compute the likelihoods for each class
    likelihoods = []
    for id in range(N_class):
        prob = 1
        for i in range(N_joints):
            for j in range(N_dims):
                prob *= log_normpdf(instances[i][j], model.jointparts[i].means[j][id], model.jointparts[i].sigma[j][id])
        likelihoods.append(prob)

    probs = [model.class_priors[label[id]] * likelihoods[id] for id in range(N_class)]
    
    return np.array(probs).T

# TEST PART

## 1 learn model

In [9]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']
# the index for training set
ind = dd['train_indexes']
[trainInd,nouse] = np.where(ind==1)
# get the training set with fixed index
X_train = X[:,:,trainInd]
y_train = Y[trainInd]
# training/learning
modelNB = learn_model(X_train, y_train)

## 2 Classify

In [10]:
dd = scipy.io.loadmat('data/validation_data.mat')
X = dd['data_small']
Y = dd['labels_small']

# get the test set index
ind = dd['test_indexes']
[testInd,nouse] = np.where(ind==1)
# get the testing set with fixed index
X_test = X[:,:,testInd]
y_test = Y[testInd]
# classify  
y_rst = classify_samples(X_test, modelNB)
labelpos = np.argmax(y_rst, axis = 1)
# get the label of the each instance with probs
classPriors = modelNB.class_priors
whatlabel = list(classPriors.keys())
y = classify_samples(X_test, modelNB)
labelpos = np.argmax(y, axis = 1)

rst = [whatlabel[labelpos[i]] for i in range(len(labelpos))]
val = list(y_test.reshape(len(y_test),))

print(cal_acc(rst,val))

0.95
